In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install rdkit

In [ ]:
!pip install duckdb

The dataset is expanded to 30,000 samples for each class.
  
Only the SVM with the RBF kernel, C=10, and gamma='auto' is trained and evaluated.
  
Learning curves and evaluation metrics are generated using 5-fold cross-validation.

### Model Training

In [ ]:
import os
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score, accuracy_score, roc_auc_score, f1_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import duckdb
import time
import joblib
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Function to generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return [0] * bits
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

# Load and preprocess data
train_path = '/kaggle/input/leash-BELKA/train.parquet'

logger.info("Connecting to DuckDB...")
con = duckdb.connect()
query = f"""
    (SELECT * FROM parquet_scan('{train_path}') WHERE binds = 0 ORDER BY random() LIMIT 10000)
    UNION ALL
    (SELECT * FROM parquet_scan('{train_path}') WHERE binds = 1 ORDER BY random() LIMIT 10000)
"""
df = con.query(query).df()
con.close()

logger.info("Data loaded and sampled...")

# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)
df['ecfp'] = df['molecule'].apply(generate_ecfp)

# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + list(protein) for ecfp, protein in zip(df['ecfp'], protein_onehot)]
y = df['binds'].values

# Split the data into train and test sets with a fixed random seed for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

logger.info("Data split into training and test sets...")

# Define a function to plot learning curves
def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, random_state=RANDOM_SEED)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    plt.legend(loc="best")
    return plt

# Support Vector Machine
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(probability=True, random_state=RANDOM_SEED, kernel='rbf', C=10, gamma='auto'))
])

logger.info("Training SVM model...")
# Train the SVM model
start_time = time.time()
svm_pipeline.fit(X_train, y_train)
logger.info(f"SVM model trained in {time.time() - start_time:.2f} seconds.")
svm_best = svm_pipeline

# Save the trained model
model_filename = 'svm_best_model.pkl'
joblib.dump(svm_best, model_filename)
logger.info(f"Model saved to {model_filename}")

# Plot learning curve for the best SVM model
plot_learning_curve(svm_best, "Learning Curve (SVM)", X_train, y_train, cv=5, n_jobs=-1)

# Evaluation Metrics
def evaluate_model(model, X_test, y_test):
    start_time = time.time()
    y_pred = model.predict(X_test)
    fit_time = time.time() - start_time

    start_time = time.time()
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    wall_clock_time = time.time() - start_time

    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred)
    avg_precision = average_precision_score(y_test, y_pred_proba)

    logger.info(f"Accuracy: {accuracy}")
    logger.info(f"ROC AUC Score: {roc_auc}")
    logger.info(f"F1 Score: {f1}")
    logger.info(f"Average Precision Score: {avg_precision}")
    logger.info(f"Fit Time: {fit_time}")
    logger.info(f"Wall Clock Time: {wall_clock_time}")

    return accuracy, roc_auc, f1, avg_precision, fit_time, wall_clock_time

logger.info("Evaluating SVM model...")
svm_results = evaluate_model(svm_best, X_test, y_test)

plt.show()

# ROC and Precision-Recall Curves
def plot_roc_curve(model, X_test, y_test, model_name):
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.plot(fpr, tpr, label=f"{model_name} (area = {roc_auc_score(y_test, y_pred_proba):.2f})")

def plot_precision_recall_curve(model, X_test, y_test, model_name):
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    plt.plot(recall, precision, label=f"{model_name} (area = {average_precision_score(y_test, y_pred_proba):.2f})")

plt.figure()
plot_roc_curve(svm_best, X_test, y_test, "SVM")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="best")
plt.show()

plt.figure()
plot_precision_recall_curve(svm_best, X_test, y_test, "SVM")
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="best")
plt.show()


### Inference

In [ ]:
import os
import numpy as np
import pandas as pd
from rdkit import Chem
from sklearn.preprocessing import OneHotEncoder
import joblib
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load the trained model
model_filename = 'svm_best_model.pkl'
svm_best = joblib.load(model_filename)
logger.info(f"Model loaded from {model_filename}")

# Function to generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return [0] * bits
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

# One-hot encoder needs to be reloaded as well
onehot_encoder = OneHotEncoder(sparse_output=False)

# Generate Submission
test_file = '/kaggle/input/leash-BELKA/test.parquet'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
df_test = pd.read_parquet(test_file)

# Generate ECFPs for the molecule_smiles
df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

# One-hot encode the protein_name
protein_onehot = onehot_encoder.fit_transform(df_test['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X_test_full = [ecfp + list(protein) for ecfp, protein in zip(df_test['ecfp'], protein_onehot)]

# Predict the probabilities
probabilities = svm_best.predict_proba(X_test_full)[:, 1]

# Create a DataFrame with 'id' and 'binds' columns
output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

# Save the output DataFrame to a CSV file
output_df.to_csv(output_file, index=False)

logger.info("Submission file created.")
